In [37]:
import dataset
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from glob import glob
import os, random
import model

In [38]:
def set_seed(seed: int = 42):
    random.seed(seed)
    #np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(0)

In [39]:
import importlib
importlib.reload(model)
importlib.reload(dataset)

<module 'dataset' from 'D:\\f1\\ch4\\dataset.py'>

In [ ]:
import model
import albumentations as A
from albumentations.augmentations.transforms import Flip
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from albumentations.augmentations.geometric.transforms import Affine


if __name__ == '__main__':
    ckpt = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor = 'val_loss',save_top_k = 1,mode = 'min')
    trainer = Trainer(max_epochs = 10,gpus = 1, callbacks = ckpt, precision=16,deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        Affine(translate_percent=0.1, p=0.5),
    ])
    
    val_tr = A.Compose([
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    
    #ckpt = torch.load(path)
    #model.load_state_dict(ckpt['state_dict'])
    trainer.fit(model)
    trainer.test(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 17.6 M
----------------------------------------
17.6 M    Trainable params
0         Non-trainable params
17.6 M    Total params
70.223    Total estimated model params size (MB)


Loaded pretrained weights for efficientnet-b4


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [33]:
import model
model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':None,'val_tr':None})

ckpt = torch.load(glob('lightning_logs/version_/checkpoints/*.ckpt')[0]) # ADD LATEST VERSION TO .../version_(here)/...
model.load_state_dict(ckpt['state_dict'])
trainer.test(model)
out = trainer.predict(model)

Loaded pretrained weights for efficientnet-b4


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 0.996999979019165, 'test_loss': 0.012221664190292358}
--------------------------------------------------------------------------------


Predicting: 1250it [00:00, ?it/s]

In [34]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i]).argmax(1)))

In [35]:
import model
model.writeSub(output)

In [36]:
!aicrowd submission create -c f1-car-rotation -f submission.csv

submission.csv ------------------- 100.0% • 115.6/113.9 KB • 1.4 MB/s • 0:00:00                                                 +-------------------------+                                                 
                                                 | Successfully submitted! |                                                 
                                                 +-------------------------+                                                 
                                                       Important links                                                       
+---------------------------------------------------------------------------------------------------------------------------+
|  This submission | https://www.aicrowd.com/challenges/ai-blitz-8/problems/f1-car-rotation/submissions/136729              |
|                  |                                                                                                        |
|  All submissions | https://www.aicrow